# **Detecção de Fraudes em Pagamentos Online**

* Utilizando a Árvore de Decisão para detecção de fraudes em pagamentos online.

### **1. Introdução**

<p align="justify">
Nos últimos anos, os pagamentos online revolucionaram a forma como realizamos transações financeiras. A comodidade e a agilidade associadas a esse método de pagamento o tornaram uma escolha preferencial para consumidores e empresas em todo o mundo. No entanto, com o aumento na popularidade dos pagamentos online, surgiram desafios significativos relacionados à segurança, em especial a ocorrência de fraudes em pagamentos online.

<p align="justify">
A detecção de fraudes em pagamentos online é um tópico de extrema importância e relevância no cenário financeiro atual. À medida que os métodos de fraude se tornam mais sofisticados, é fundamental que as empresas e instituições financeiras desenvolvam abordagens igualmente avançadas para detectar e prevenir essas atividades fraudulentas.

<p align="justify">
Ao longo deste projeto, exploraremos as diversas facetas da detecção de fraudes em pagamentos online, desde a coleta e preparação de dados até a implementação de modelos de Machine Learning e a validação de seu desempenho.

In [ ]:
# Carregando bibliotecas

import numpy as np
import pandas as pd
import plotly.express as px

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [ ]:
# Carregando base de dados

dados = pd.read_csv('/content/drive/MyDrive/Ciência de Dados com Python/Projetos/Dados/Detecção de Fraudes.csv', sep = ',')

In [ ]:
# Visualizando base de dados

dados.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


### **2. Análise Exporatória de Dados**

In [ ]:
# Verificando shape dos dados

dados.shape

(6362620, 11)

In [ ]:
# Verificando a existência de valores faltantes

dados.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [ ]:
# Verificando informações sobre o dataset

dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [ ]:
# Análisando os tipos de transações

dados.type.value_counts()

CASH_OUT    2237500
PAYMENT     2151495
CASH_IN     1399284
TRANSFER     532909
DEBIT         41432
Name: type, dtype: int64

In [ ]:
# Gráfico 1: distribuição das transações

## Selecionando informações
tipo = dados["type"].value_counts()
transacoes = tipo.index
quantidade = tipo.values

## Plotando gráfico
figure = px.pie(dados,
                values = quantidade,
                names = transacoes,
                hole = 0.5)
figure.update_layout(title_text = "Distribuição dos Tipos de Transações",
                     title_x = 0.5)
figure.show();

In [ ]:
# Verificando a correlação entre as variáveis

dados.corr(numeric_only = True)

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
step,1.000000,0.022373,-0.010058,-0.010299,0.027665,0.025888,0.031578,0.003277
amount,0.022373,1.000000,-0.002762,-0.007861,0.294137,0.459304,0.076688,0.012295
oldbalanceOrg,-0.010058,-0.002762,1.000000,0.998803,0.066243,0.042029,0.010154,0.003835
newbalanceOrig,-0.010299,-0.007861,0.998803,1.000000,0.067812,0.041837,-0.008148,0.003776
oldbalanceDest,0.027665,0.294137,0.066243,0.067812,1.000000,0.976569,-0.005885,-0.000513
newbalanceDest,0.025888,0.459304,0.042029,0.041837,0.976569,1.000000,0.000535,-0.000529
isFraud,0.031578,0.076688,0.010154,-0.008148,-0.005885,0.000535,1.000000,0.044109
isFlaggedFraud,0.003277,0.012295,0.003835,0.003776,-0.000513,-0.000529,0.044109,1.000000


**Obs:** com base nas correlações iremos selecionar *type*, *amount*, *oldbalanceOrg*, *newbalanceOrig*.

In [ ]:
# Realizando transformações na coluna type: de categórico para numérico

dados["type"] = dados["type"].map({"CASH_OUT": 1, "PAYMENT": 2,
                                   "CASH_IN": 3, "TRANSFER": 4,
                                   "DEBIT": 5})

In [ ]:
# Verificando transformações

dados['type'].unique()

array([2, 4, 1, 5, 3])

In [ ]:
# Realizando transformações na coluna isFraud: de numérico para categórico

dados["isFraud"] = dados["isFraud"].map({0: "No Fraud", 1: "Fraud"})

In [ ]:
# Verificando transfomações

dados['isFraud'].unique()

array(['No Fraud', 'Fraud'], dtype=object)

### **4.Construindo Modelo**

In [ ]:
# Separando os dados em previsores e classes

previsores = np.array(dados[["type", "amount", "oldbalanceOrg", "newbalanceOrig"]])
classes = np.array(dados['isFraud'])

In [ ]:
# Separando os dados em treino e teste

## 70% para treino e 30% para teste
X_treino, X_teste, y_treino, y_teste = train_test_split(previsores,
                                                        classes,
                                                        test_size = 0.3,
                                                        random_state = 0)

In [ ]:
# Verificando distribuição dos dados

print(X_treino.shape)
print(y_treino.shape)
print(X_teste.shape)
print(y_teste.shape)

(4453834, 4)
(4453834,)
(1908786, 4)
(1908786,)


In [ ]:
# Instânciando modelo

modelo = DecisionTreeClassifier()

In [ ]:
# Treinando modelo

modelo.fit(X_treino, y_treino)

DecisionTreeClassifier()

In [ ]:
# Aplicando modelo aos dados de teste: realizando previsões

y_predito = modelo.predict(X_teste)

### **5. Avaliação do Modelo**

In [ ]:
# Métricas do modelo

print(classification_report(y_teste, y_predito))

              precision    recall  f1-score   support

       Fraud       0.88      0.88      0.88      2419
    No Fraud       1.00      1.00      1.00   1906367

    accuracy                           1.00   1908786
   macro avg       0.94      0.94      0.94   1908786
weighted avg       1.00      1.00      1.00   1908786



In [ ]:
# Aplicando modelo a novos dados

new_data = np.array([[4, 9000.60, 9000.60, 0.0]])
modelo.predict(new_data)

array(['Fraud'], dtype=object)

### **6. Conclusão**

<p align="justify">
Após cumprir as etapas de Análise Exploratória e Pré-Processamento dos dados, foi construido um modelo de Árvore de Decisão. Como resultado da construção desse modelo, verificou-se que o mesmo apresenta um bom desempenho com base nas métricas e resultados apresentados. Apesar da classe Fraud ter um número bem menor de dados em relação a No Fraud o modelo conseguiu apresentar resultados satisfatórios para o estudo.